In [3]:
# 남은 할일 
# 기사 제목 끌고 와서 라벨링 해준다
# 그 데이터 가지고 정확도를 예측한다. 이 때 사용될 건 accuracy, f1 score 

import pandas as pd
from konlpy.tag import Hannanum  # Hannanum 형태소 분석기 불러오기
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

search = input("회사명(csv 파일명) : ")
num = int(input("몇 번째 테스트? "))
# csv
df = pd.read_csv(f'csv/{search} 뉴스 제목{num}.csv',encoding="utf-8")

# df = pd.read_csv(f'csv/kakao_data_label.csv',encoding='cp949') # encoding='cp949'
# KoELECTRA 모델 로드
model_name = "hyunwoongko/kobart"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 형태소 분석 함수
hannanum = Hannanum()  # Hannanum 형태소 분석기 객체 생성
def tokenize(text):
    return hannanum.morphs(text)

# df['content'] = df['title']

# title 열에 대해 형태소 분석 적용

df['title'] = df['title'].astype(str).apply(tokenize)
# for i in range(df.shape[0]):
#     df.iloc[i,1] = df.iloc[i,1].apply(tokenize)


# 감성 분석을 위한 전처리 함수
def preprocess(text):
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
    inputs.to(device)
    return inputs

# 예측 함수
def predict(inputs):
    outputs = model(**inputs)
    logits = outputs.logits
    probs = logits.softmax(dim=-1)
    return probs[0].detach().cpu().numpy()

# title을 예측해서 수치화시켜 sentiment으로 저장. 즉, title를 수치화 시킨 것이 sentiment
df['sentiment'] = df['title'].apply(lambda x: predict(preprocess(' '.join(x))))

# 0.5 기준으로 하면 부정확하긴 함... 정확도를 높이려면 이 부분 건들면 좋을 듯 또는 중립을 포함시키는 것도 해봐야 할 듯
def convert_sentiment(probs):
    if probs[0] < 0.5:
        return 0
    elif probs[0]>= 0.5:
        return 1
#     else:
#         return '중립'

# train할 label은 제목을 읽고 내가 직접 라벨링
# test할 label은 0.5를 기준으로 sentiment가 0.5보다 크면 1, 작으면 0으로 기준 세움
df['label'] = df['sentiment'].apply(convert_sentiment)
df['sentiment'] = df['sentiment'].apply(lambda x: x[0]).tolist()
df=round(df,2)

회사명(csv 파일명) : 삼성전자
몇 번째 테스트? 9


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
Some weights of BartForSequenceClassification were not initialized from the model checkpoint at hyunwoongko/kobart and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [4]:
# csv 파일로 저장
df.to_csv(f'csv/sentiment_{search}_{num}.csv', index=False)

# 결과 확인
print(df['title'])
print(df['sentiment'])

0               [삼성전자, ,, 윈도, 운영체제, 탑재한, 키오스크, 신제품, 출시]
1                 [삼성전자, ,, 윈도우, 탑재한, ', 삼성, 키오스크', 출시]
2     [네이처, 에, 실리, ㄴ, 삼성전자, ', 강유전체, ', 논문, 언박싱, &lt...
3                  [기재부, ,, 넥슨, 2대주주, 등극에…삼성家, 사례, 재조명]
4     [이건희, 가, 만들, ㄴ, ', 품질, 의, 삼성'…JY, ', 초일류, 삼성, ...
5          [삼성家, 세, 모녀, ,, 상속세, 마련, 위하, 어, 4조원, 이상, 대출]
6        [애플, ,, MR, 헤드셋, 전격, 공개, ..., 메타·삼성, 등, 과, 경쟁]
7      [“TSMC, ‘2나노, 반도체’, 시범생산, 착수…, 삼성, 에, 크, ㄴ, 압박”]
8     ["반도체·디스플레이·AI·바이오, 에, 투, 이, 자, 역량, 집중해, 이, 야,...
9     [팀쿡, 의, ‘원, 모으, 아, 띵’, 은, ‘비전, 프로’…‘공간컴퓨팅’, 내세...
10                  [일본, 은, 어쩌다, 반도체, 추락, 국가, 가, 되, 었나]
11    [삼성, 신경영, 30년, ,, 이제, 는, ', 이재용, 회장, ', 의, 시간,...
12    [메타, 이, 이, 어, 애플·삼성, 도, 가, 세, ...', 메타버스, 헤드셋'...
13    [', 초일류, 삼성, ', 을, 만들, ㄴ, 30년, 전, 이건희, 의, 한, 마...
14    [40년, 전, 삼성, 제안, 에, 뒤바뀌, ㄴ, 운명…'세계, 1위, ', 오르,...
15    [“1만대, 모두, 완판입니다”, 망, 하, ㄴ, 줄, 알, 아ㄴ데, ,, 이런일이...
16       [삼성·LG,, "친환경, 라운지, 열, 고, ,, 흙공, 던지, 기, 집중, "]
17    [외국인, 은, 반도체, ', 폭풍, 매수'…개미, 는, 2차전지, 에, '

In [ ]:
df